In [104]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Thu Oct 24

@author: yaning
"""

import Receptors as Receptors
import importlib
import numpy as np
import matplotlib.pyplot as plt

importlib.reload(Receptors)

<module 'Receptors' from '/home/yaning/Documents/Spiking_NN/Receptors.py'>

In [105]:
class Neuron:
    _Cm = 1

    def __init__(self, deltaTms, I, Vm):
        self.deltaTms = deltaTms
        self.I = I
        self.Vm = Vm
        self.incoming_synapses = []
        self.outgoing_synapses = []

    
    def inject_current(self, sum_currents):
        self.I += sum_currents
    
    def update_voltage(self):
        self.Vm += - self.deltaTms * self.I / self._Cm
    
    def sending_signal(self):
        for synapse in self.outgoing_synapses:
            synapse.state = 1


    # def add_incoming_synapses(self, ):
    #     self.incoming_synapses.append()

In [106]:
class Synapse:

    def __init__(self, deltaTms, state, send_neuron, receive_neuron, *args):
        # either it is being simulated 1 or not 0
        self.deltaTms = deltaTms
        self.state = state

        # do we need send neurons?
        self.send_neuron = send_neuron
        self.receive_neuron = receive_neuron

        self.receptors = []

        if args:
            self.receptors.extend(args)
    

    def update(self):
        sum_currents = 0
        for receptor in self.receptors:
            receptor.update_gP(self.state, self.deltaTms)
            sum_currents += receptor.current()
        
        self.receive_neuron.inject_current(sum_currents)


In [107]:
class Control:
    def __init__(self, deltaTms, Vm):
        self.all_synapses = []
        self.deltaTms = deltaTms
        self.Vm = Vm

    def create_synapse(self, send_neuron, receive_neuron, type):
        sodium_channel = Receptors.Voltage_Sodium(self.Vm)
        potassium_channel = Receptors.Voltage_Potassium(self.Vm)
        leaky_channel = Receptors.Voltage_Leak(self.Vm)
        
        if type == "AMPA":
            ampa_receptor = Receptors.LigandGatedChannelFactory.create_AMPA(self.Vm)
            synapse = Synapse(self.deltaTms, 0, send_neuron, receive_neuron, ampa_receptor)
            
        elif type == "AMPA+NMDA":
            ampa_receptor = Receptors.LigandGatedChannelFactory.create_AMPA(self.Vm)
            nmda_receptor = Receptors.LigandGatedChannelFactory.create_NMDA(self.Vm)
            synapse = Synapse(self.deltaTms, 0, send_neuron, receive_neuron, ampa_receptor, nmda_receptor)
        
        elif type == "GABA":
            gaba_receptor = Receptors.LigandGatedChannelFactory.create_GABA(self.Vm)
            synapse = Synapse(self.deltaTms, 0, send_neuron, receive_neuron, gaba_receptor)

        send_neuron.outgoing_synapses.append(synapse)
        receive_neuron.incoming_synapses.append(synapse)

        self.all_synapses.append(synapse)
            
            

In [108]:
pointCount = 5000
deltaTms = 0.05
times = np.arange(pointCount) * deltaTms
initial_Vm = 1.3458754117369027

neuron1 = Neuron(deltaTms, 0, -65)
neuron2 = Neuron(deltaTms, 0, -65)

control = Control(deltaTms, initial_Vm)
control.create_synapse(neuron1, neuron2, "AMPA")

for t in range(pointCount):
    for synapse in control.all_synapses:
        synapse.update()

In [109]:
neuron1.outgoing_synapses[0].receptors